In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib notebook

import os
import glob
import math
import pandas as pd
import numpy as np
import itertools
import random

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Layer,concatenate, Conv2D, BatchNormalization, Concatenate, UpSampling2D, MaxPool2D, Input, Dropout, Dense, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow.keras.backend as K

from IPython.display import SVG

import tensorflow_probability as tfp


from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, normalize
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')

from ipywidgets import FloatProgress
from IPython.display import display

import HAR_utils
from HAR_utils import get_activity_index, plot_confusion_matrix, get_Train_Test, timeseries_standardize
from attention import Attention2D
from getattention import get_intermediate_output, write_intermediate_output

fig_width = 12
plt.rcParams["font.size"] = 7

## Load Data

In [ ]:
data_df = pd.read_csv(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\daily_data\acc_03_r.csv')

In [ ]:
#data_df = pd.read_csv(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\daily_data\acc_04_r.csv')

In [ ]:
#data_df = pd.read_csv(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\daily_data\acc_05_r.csv')

In [ ]:
#data_df = pd.read_csv(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\daily_data\acc_06_r.csv')

In [ ]:
data_df.drop(['LW_x','LW_y','LW_z'],axis=1,inplace=True)

## Transform Data

In [ ]:
trainX,testX,trainY,testY,testtimes,selector = get_Train_Test(data_df,trn=37)

In [ ]:
print(selector)

In [ ]:
test_data = data_df[data_df['job']==selector]

In [ ]:
test_data

In [ ]:
print('trainX(shape): ',trainX.shape)
print('trainY(shape): ',trainY.shape)

print('testX(shape): ',testX.shape)
print('testY(shape): ',testY.shape)
print('testtimes(len): ',len(testtimes))

In [ ]:
tex = trainX[[5]]
trx = np.delete(trainX,5,0)
print('trainX(shape): ',tex.shape)
print('trainX(shape): ',trx.shape)

In [ ]:
trainX , testX = timeseries_standardize(trainX, testX)

# Network design

In [ ]:
def attention_block(ip,filters=10):
    depth_k = 10
    depth_v = 10
    num_heads = 1
    qkv_conv = Conv2D(2 * depth_k + depth_v, (1, 1), strides=(1, 1), padding='same', use_bias=True, kernel_initializer='he_normal', name="pre-qkvcon")(ip)
    attn_out = Attention2D(depth_k, depth_v, num_heads, True, name="attn")(qkv_conv)
    attn_out = Conv2D(depth_v, (1, 1), strides=(1, 1), padding='same', use_bias=True, kernel_initializer='he_normal', name="pos-qkvcon")(attn_out)
    att_output = Multiply()([ip, attn_out])
    att_output = BatchNormalization()(att_output)
    return att_output

def down_block(x, filters, kernel_size=(1, 3), padding="same", strides=1, name="down_"):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    p = MaxPool2D((1, 2),name=name+"M")(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(1, 3), padding="same", strides=1, name="up_"):
    us = UpSampling2D((1, 2),name=name+"U")(x)
    concat = Concatenate()([us, skip])
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(concat)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    return c

def bottleneck(x, filters, kernel_size=(1, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl2")(c)
    c = Dropout(0.5)(c)
    return c

def build_unet_model(height,channels,class_num,f=32):
    
    inputs = Input((1, height, channels))
    
    p0 = inputs
    c1, p1 = down_block(p0, f, name="down_1_")
    att1 = attention_block(c1, filters=f)
    pa = MaxPool2D((1, 2))(att1)
    c2, p2 = down_block(pa, f*2, name="down_2_")
    c3, p3 = down_block(p2, f*4, name="down_3_")
    p3 = Dropout(0.5)(p3)
    
    bn = bottleneck(p3, f*8)
    
    u1 = up_block(bn, c3, f*4, name="up_1_")
    u2 = up_block(u1, c2, f*2, name="up_2_")
    u3 = up_block(u2, att1, f, name="up_3_")
    
    outputs = Dense(class_num, activation = 'softmax')(u3)
    model = Model(inputs, outputs)
    
    return model

def ApplyUnet(trainX, trainY, testX, testY,TestTimes, activities, nb_epoch=100, fi=32):
    
    target_names,indices = get_activity_index(activities)
    #print(target_names)
    indices.append(11)
    target_names.append('EPadding')
    
    trainY_b = np_utils.to_categorical(trainY, len(target_names))
    testY_b = np_utils.to_categorical(testY, len(target_names))
    lb = LabelBinarizer()
    
    print(indices)
    lb.fit(indices)
    
    model = build_unet_model(trainX.shape[2],trainX.shape[3],len(target_names),f=fi)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    model.summary()
    
    #trainX = tf.convert_to_tensor(trainX, dtype=tf.float32)
    #trainY_b = tf.convert_to_tensor(trainY_b, dtype=tf.float32)
    def scheduler(epoch,lr):
        if epoch < 20:
            return lr
        else:
            return float(lr)
        
    callback = keras.callbacks.LearningRateScheduler(scheduler)
    model_checkpoint = keras.callbacks.ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    history = model.fit(trainX, trainY_b, epochs=nb_epoch, validation_split=0.2, batch_size = 2, callbacks=[callback])
    
    fig, (tr_plt,cm_plt) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6))
    
    tr_plt.plot(history.history['loss'])
    #tr_plt.plot(history.history['val_loss'])
    tr_plt.set_title('Model loss')
    tr_plt.set_ylabel('Loss')
    tr_plt.set_xlabel('Epoch')
    tr_plt.legend(['loss', 'val_loss'], loc='upper left')
      
    predict_v = model.predict(testX)
    predict_vec = predict_v[0][0]
    confidences = np.max(predict_vec, axis=1)
    testYY = testY[0][0]
    
    predict_class = np.argmax(predict_vec, axis=1)
    predict = [lb.classes_[x] for x in predict_class]
    
    print("len(testYY)",len(testYY))
    print("len(predict)",len(predict))
    
    testXX = testX[0][0]
    
    '''
    for i in range(len(testYY)):
        if testYY[i]==11:
            testYYY = testYY[:i]
            predict1 = predict[:i]
            testXXX = testXX[:i]
            ttimes = TestTimes[:i]
            Confidences = confidences[:i]
            print(i)
            break
    
    '''
    testYYY = testYY
    predict1 = predict
    testXXX = testXX
    ttimes = TestTimes
    Confidences = confidences
    
    
    print("len(testYYY)",len(testYYY))
    print("len(predict1)",len(predict1))
    
    result = classification_report(testYYY, predict1, labels=indices, target_names=target_names)
    print(result)
    testXX=testX
    cnf_matrix = confusion_matrix(testYYY, predict1)
    plot_confusion_matrix(cnf_matrix, fig, cm_plt, classes=target_names)
    fig.tight_layout()
    fig.show()

    return predict1, testYYY, testXXX, Confidences, ttimes, model

## Network testing

In [ ]:
simclass_df = pd.read_csv(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\TF-env\projects\U-net\acc2\labels.csv')
simclass_d = simclass_df['activity'].to_dict()
#simclass_d[11]='pad_class'

In [ ]:
tf.config.experimental_run_functions_eagerly(True)

In [ ]:
predict1, testYYY, testXXX, Confidences, Ttimes, model = ApplyUnet(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=100, fi=10)

In [ ]:
normal_files = []
for i in range(len(trainX)):
    file = 'layer_output_res_4_1_' + str(i) + '.csv'
    normal_files.append(file)

In [ ]:
write_intermediate_output(model, trainX, 4, normal_files, r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-17-08\Self-Att-con\Layer_values', Ttimes, layers=['attn'])

In [ ]:
test_data_with_conf_pred = pd.DataFrame(data = {'time':Ttimes, 'l_val':testYYY, 'predicted_label':predict1,'confidence':Confidences})

In [ ]:
#test_data_with_conf_pred[test_data_with_conf_pred['predicted_label']==8]

In [ ]:
test_data_with_conf_pred.to_csv(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-17-08\Self-Att-con\6\res_6_10.csv',index=False)

In [ ]:
tttimes = range(len(predict1))

In [ ]:
labels,indices = get_activity_index(simclass_d)
indices.append(11)
labels.append('EPadding')
figs = plt.figure('result',figsize=(10, 4))
axs = plt.subplot(111)
axs.plot(tttimes, predict1, color='r')
axs.plot(tttimes, testYYY, color='g')
axs.set_ylabel('Activity label')
axs.set_yticks(indices)
axs.set_yticklabels(labels)
figs.show()